<a href="https://colab.research.google.com/github/unVeroz007/DIF62130_KELAS-B_25_2311532008/blob/main/Praktikum3/BernoulliNB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import library yang diperlukan

In [ ]:
import pandas as pd #Untuk membaca dan mengolah data dalam bentuk tabel.
from sklearn.naive_bayes import BernoulliNB #Algoritma Naïve Bayes untuk data biner (0 dan 1).
from sklearn.preprocessing import StandardScaler #Untuk menstandarisasi (menormalkan) data supaya model lebih akurat.
from sklearn.metrics import accuracy_score #Untuk mengukur seberapa akurat prediksi model.
from sklearn.model_selection import train_test_split #Untuk membagi data menjadi data latih (train) dan data uji (test).

#Membuat dan mengelola dataset

In [ ]:
#Membuat Data dalam Bentuk Dictionary
dataset_dict = {
    'Outlook': ['sunny', 'sunny', 'overcast', 'rainy', 'rainy', 'rainy', 'overcast', 'sunny', 'sunny', 'rainy', 'sunny', 'overcast', 'overcast', 'rainy', 'sunny', 'overcast', 'rainy', 'sunny', 'sunny', 'rainy', 'overcast', 'rainy', 'sunny', 'overcast', 'sunny', 'overcast', 'rainy', 'overcast'],
    'Temperature': [85.0, 80.0, 83.0, 70.0, 68.0, 65.0, 64.0, 72.0, 69.0, 75.0, 75.0, 72.0, 81.0, 71.0, 81.0, 74.0, 76.0, 78.0, 82.0, 67.0, 85.0, 73.0, 88.0, 77.0, 79.0, 80.0, 66.0, 84.0],
    'Humidity': [85.0, 90.0, 78.0, 96.0, 80.0, 70.0, 65.0, 95.0, 70.0, 80.0, 70.0, 90.0, 75.0, 80.0, 88.0, 92.0, 85.0, 75.0, 92.0, 90.0, 85.0, 88.0, 65.0, 70.0, 60.0, 95.0, 70.0, 78.0],
    'Wind': [False, True, False, False, False, True, True, False, False, False, True, True, False, True, True, False, False, True, False, True, True, False, True, False, False, True, False, False],
    'Play': ['No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No', 'Yes', 'Yes', 'No', 'No', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'Yes']
}

df = pd.DataFrame(dataset_dict) #Mengubah Dictionary Menjadi DataFrame

df.head(10) #menampilkan 10 data pertama

,Outlook,Temperature,Humidity,Wind,Play
0,sunny,85.0,85.0,False,No
1,sunny,80.0,90.0,True,No
2,overcast,83.0,78.0,False,Yes
3,rainy,70.0,96.0,False,Yes
4,rainy,68.0,80.0,False,Yes
5,rainy,65.0,70.0,True,No
6,overcast,64.0,65.0,True,Yes
7,sunny,72.0,95.0,False,No
8,sunny,69.0,70.0,False,Yes
9,rainy,75.0,80.0,False,Yes


#Melakukan pengolahan data
pd.get_dummies(df, columns=['Outlook']) berguna untuk Mengubah kolom "Outlook" (yang berisi kategori: "sunny", "overcast", "rainy") menjadi beberapa kolom baru dengan nilai 0 atau 1

prefix='' agar Tidak menambahkan awalan pada nama kolom baru.

prefix_sep='' untuk Tidak menambahkan pemisah antara awalan dan nama asli.

dtype=int untuk Memastikan hasil encoding dalam tipe data integer (0 dan 1).

In [ ]:
#one hot encode 'Outlook' Column
df = pd.get_dummies(df, columns=['Outlook'], prefix='', prefix_sep='', dtype=int)

#CONVERT 'Windy' (bool) and 'Play' (binary) COLUMNS TO BINARY INDICATORS
df['Wind'] = df['Wind'].astype(int) #Mengubah Kolom "Wind" Menjadi Integer

df['Play'] = (df['Play']=='Yes').astype(int) # Mengecek apakah nilai dalam kolom "Play" adalah "Yes" (menghasilkan True/False). astype(int) untuk mengubah "Yes" menjadi 1, "NO" menjadi 0

df.head() #menampilkan 5 data awal

,Temperature,Humidity,Wind,Play,overcast,rainy,sunny
0,85.0,85.0,0,0,0,0,1
1,80.0,90.0,1,0,0,0,1
2,83.0,78.0,0,1,1,0,0
3,70.0,96.0,0,1,0,1,0
4,68.0,80.0,0,1,0,1,0


#Melakukan pengelompokan (binning) dan one-hot encoding pada kolom "Temperature" dan "Humidity", lalu menambahkannya kembali ke DataFrame dalam bentuk numerik.

##Mengelompokkan Data "Temperature" dan "Humidity" (Binning)
pd.cut() digunakan untuk mengelompokkan nilai numerik ke dalam kategori.

bins=[0, 80, 100] → Membagi nilai Temperature menjadi:
0 - 80 → "Warm"
81 - 100 → "Hot"

bins=[0, 75, 100] → Membagi nilai Humidity menjadi:
0 - 75 → "Dry"
76 - 100 → "Humid"

##Menggunakan One-Hot Encoding untuk Kolom "Temperature" dan "Humidity"
pd.get_dummies() mengubah kolom kategori menjadi 0 dan 1 (one-hot encoding).

drop_first=True
→ Menghapus salah satu kategori untuk menghindari multikolinearitas.

Misalnya, jika ada ["Warm", "Hot"], maka hanya "Hot" yang disimpan:
Hot = 1 → Panas
Hot = 0 → Warm

Sama juga untuk ["Dry", "Humid"]:
Humid = 1 → Lembab
Humid = 0 → Kering

dtype=int mengonversi hasil encoding menjadi integer (0/1).

In [ ]:
#One-hot encode the categories columns and drop them after
#define categories fot "Temperature" and "Humidity" for dataFrame
#Mengelompokkan Data "Temperature" dan "Humidity" (Binning)
df['Temperature'] = pd.cut(df['Temperature'], bins=[0, 80, 100], labels=['Warm', 'Hot'])
df['Humidity'] = pd.cut(df['Humidity'], bins=[0, 75, 100], labels=['Dry', 'Humid'])

#One-hot encode the categorized columns
#Menggunakan One-Hot Encoding untuk Kolom "Temperature" dan "Humidity"
one_hot_columns = pd.get_dummies(df[['Temperature', 'Humidity']], drop_first=True, dtype=int)

#drop the categorized columns from df
#Menghapus Kolom Asli Setelah One-Hot Encoding
df = df.drop(columns=['Temperature', 'Humidity'], axis=1) #Kolom asli "Temperature" dan "Humidity" dihapus karena sudah dikonversi.

#Concatenate the one-hot encoded columns with the original DataFrame
#Menggabungkan One-Hot Encoded Data dengan DataFrame Awal
df = pd.concat([df, one_hot_columns], axis=1) #Menggabungkan hasil one-hot encoding kembali ke DataFrame.

df.head() #Menampilkan 5 data awal

,Wind,Play,overcast,rainy,sunny,Temperature_Hot,Humidity_Humid
0,0,0,0,0,1,1,1
1,1,0,0,0,1,0,1
2,0,1,1,0,0,1,1
3,0,1,0,1,0,0,1
4,0,1,0,1,0,0,1


#Membagi dataset menjadi data latih (training set) dan data uji (testing set) agar bisa digunakan dalam model machine learning.

##X, y = df.drop(columns='Play'), df['Play']
df.drop(columns='Play')
→ Menghapus kolom "Play" dari dataset untuk mendapatkan X (fitur).

df['Play']
→ Mengambil kolom "Play" sebagai y (label/target yang ingin diprediksi).

##train_test_split(X, y, test_size=0.3, shuffle=False):
test_size=0.3 → 30% data digunakan sebagai data uji, 70% sisanya sebagai data latih.

shuffle=False → Data tidak diacak, sehingga urutannya tetap seperti di dataset awal.

In [ ]:
#Split data info training and testing
#Memisahkan Fitur (X) dan Label (y)
X, y = df.drop(columns='Play'), df['Play']

#Membagi Data Menjadi Data Latih dan Data Uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)

X_train.head() #Menampilkan 5 data awal

,Wind,overcast,rainy,sunny,Temperature_Hot,Humidity_Humid
0,0,0,0,1,1,1
1,1,0,0,1,0,1
2,0,1,0,0,1,1
3,0,0,1,0,0,1
4,0,0,1,0,0,1


#Menghitung probabilitas kemunculan setiap kategori dalam y_train menggunakan nilai pecahan (Fraction) agar hasilnya lebih presisi.

##Menggunakan value_counts() untuk Menghitung Frekuensi Tiap Kategori
attr.value_counts() menghitung jumlah kemunculan setiap nilai dalam y_train.

.sum() memastikan kita mendapatkan total jumlah sampel.

##Menghitung Probabilitas dengan Fraction
Fraction(x, total_counts) mengubah jumlah kemunculan setiap nilai menjadi pecahan.

.limit_denominator() memastikan pecahan dalam bentuk yang paling sederhana.

##Menjalankan Fungsi dengan y_train
Fungsi dijalankan untuk menghitung probabilitas tiap kategori dalam y_train.


In [ ]:
from fractions import Fraction #import fraction

def cal_target_prob(attr):
  total_counts = attr.value_counts().sum() #Menghitung Frekuensi Tiap Kategori
  prob_series = attr.value_counts().apply(lambda x: Fraction(x, total_counts).limit_denominator()) #Menghitung Probabilitas dengan Fraction
  return prob_series

print(cal_target_prob(y_train)) ###Menjalankan Fungsi dengan y_train

Play
1    11/19
0     8/19
Name: count, dtype: object


#Fungsi hitung_rasio() untuk Menghitung Proporsi
##def hitung_rasio(kejadian, total):
  ##  return kejadian / total

Fungsi ini membagi jumlah kejadian (misalnya, jumlah "1") dengan total data.

Digunakan untuk menghitung rasio kemunculan nilai dalam setiap kolom.

##Menghitung Rasio 1 dan 0 untuk Setiap Kolom
X_train['Temperature_Hot'].value_counts()[1]
→ Menghitung jumlah 1s dalam kolom "Temperature_Hot".

X_train.value_counts().sum()
→ Menghitung total jumlah baris dalam dataset.

Hal yang sama dilakukan untuk kategori lain, seperti Humidity_Humid, Wind, overcast, rainy, sunny.

##Menampilkan Hasil dengan print()





In [ ]:
#menghitung rasio kemunculan nilai dalam setiap kolom.
def hitung_rasio(kejadian, total):
  return kejadian / total #Fungsi ini membagi jumlah kejadian (misalnya, jumlah "1") dengan total data.

temp_one = hitung_rasio(X_train['Temperature_Hot'].value_counts()[1], X_train.value_counts().sum())
print(f"Ratio of 1s in 'Temperature_Hot' column: {temp_one}")
temp_zero = hitung_rasio(X_train['Temperature_Hot'].value_counts()[0], X_train.value_counts().sum())
print(f"Ratio of 0s in 'Temperature_Hot' column: {temp_zero}")
print(" ")

humid_one = hitung_rasio(X_train['Humidity_Humid'].value_counts()[1], X_train.value_counts().sum())
print(f"Ratio of 1s in 'Humidity_Hot' column: {humid_one}")
humid_zero = hitung_rasio(X_train['Humidity_Humid'].value_counts()[0], X_train.value_counts().sum())
print(f"Ratio of 0s in 'Humidity_Hot' column: {humid_zero}")
print(" ")

wind_one = hitung_rasio(X_train['Wind'].value_counts()[1], X_train.value_counts().sum())
print(f"Ratio of 1s in 'Wind' column: {wind_one}")
wind_zero = hitung_rasio(X_train['Wind'].value_counts()[0], X_train.value_counts().sum())
print(f"Ratio of 0s in 'Wind' column: {wind_zero}")
print(" ")

overcast_one = hitung_rasio(X_train['overcast'].value_counts()[1], X_train.value_counts().sum())
print(f"Ratio of 1s in 'Wind' column: {overcast_one}")
overcast_zero = hitung_rasio(X_train['overcast'].value_counts()[0], X_train.value_counts().sum())
print(f"Ratio of 0s in 'overcast' column: {overcast_zero}")
print(" ")

rainy_one = hitung_rasio(X_train['rainy'].value_counts()[1], X_train.value_counts().sum())
print(f"Ratio of 1s in 'rainy' column: {rainy_one}")
rainy_zero = hitung_rasio(X_train['rainy'].value_counts()[0], X_train.value_counts().sum())
print(f"Ratio of 0s in 'rainy' column: {rainy_zero}")
print(" ")

sunny_one = hitung_rasio(X_train['sunny'].value_counts()[1], X_train.value_counts().sum())
print(f"Ratio of 1s in 'sunny' column: {sunny_one}")
sunny_zero = hitung_rasio(X_train['sunny'].value_counts()[0], X_train.value_counts().sum())
print(f"Ratio of 0s in 'sunny' column: {sunny_zero}")

Ratio of 1s in 'Temperature_Hot' column: 0.2631578947368421
Ratio of 0s in 'Temperature_Hot' column: 0.7368421052631579
 
Ratio of 1s in 'Humidity_Hot' column: 0.6842105263157895
Ratio of 0s in 'Humidity_Hot' column: 0.3157894736842105
 
Ratio of 1s in 'Wind' column: 0.42105263157894735
Ratio of 0s in 'Wind' column: 0.5789473684210527
 
Ratio of 1s in 'Wind' column: 0.2631578947368421
Ratio of 0s in 'overcast' column: 0.7368421052631579
 
Ratio of 1s in 'rainy' column: 0.3157894736842105
Ratio of 0s in 'rainy' column: 0.6842105263157895
 
Ratio of 1s in 'sunny' column: 0.42105263157894735
Ratio of 0s in 'sunny' column: 0.5789473684210527


#melatih model Naïve Bayes Bernoulli menggunakan dataset X_train dan y_train.

Model menghitung probabilitas P(Play=Yes | fitur tertentu) dan P(Play=No | fitur tertentu) menggunakan teorema Bayes.

Karena ini BernoulliNB, setiap fitur dianggap memiliki dua kemungkinan nilai: 0 atau 1.

Model menggunakan likelihood berdasarkan frekuensi dalam dataset untuk memperkirakan probabilitas kelas.

In [ ]:
from sklearn.naive_bayes import BernoulliNB #import Algoritma Naive Bayes Bernouli

nb_clf = BernoulliNB() #Membuat Model Bernoulli Naïve Bayes

nb_clf.fit(X_train, y_train) #Melatih model menggunakan data X_train (fitur) dan y_train (label). Model akan menghitung probabilitas tiap fitur untuk masing-masing kelas (Play = 1 atau Play = 0).

BernoulliNB()

#Menguji performa model Naïve Bayes Bernoulli dengan membandingkan prediksi model terhadap data uji dan menghitung akurasi.

nb_clf.predict(X_test) → Model menggunakan data uji (X_test) untuk membuat prediksi.

y_pred → Berisi hasil prediksi model (nilai 0 atau 1).

accuracy_score(y_test, y_pred) → Menghitung persentase prediksi yang benar.

Hasilnya ditampilkan dalam bentuk angka desimal antara 0 dan 1.

In [ ]:
y_pred = nb_clf.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}") #Menghitung Akurasi Model

Accuracy: 0.5555555555555556
